In [ ]:
# Eliminate outlier days? 
# Which day of the week has most number of delays and longest duration of delays?
# What are the most common reasons for delays?
# Number of delays and duration of delays over the course of a day
# Which stations are the hot spots?
# Seasonality in number and duration of delays - over a day, week, year
# Simulation? model movement of a train over a line, with delay simulated by some wait time distribution?

# TTC Subway Delay Data Analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from datetime import datetime
import seaborn as sns
import re

## Load Data

In [7]:
df = pd.read_csv('processed_data/ttc_subway_delay_data.csv', parse_dates = [0])

print(df.shape)
df.head()

(441268, 11)


,date,time,weekday,hour,line_cleaned,station_cleaned,code,desc,min_delay,indicator_ns,indicator_station
0,2014-01-02,11:30,Thursday,11,YU,VAUGHAN MC STATION,MUGD,Miscellaneous General Delays,0.013158,1,1
1,2014-01-02,11:30,Thursday,11,YU,HIGHWAY 407 STATION,MUGD,Miscellaneous General Delays,0.013158,0,1
2,2014-01-02,11:30,Thursday,11,YU,PIONEER VILLAGE STATION,MUGD,Miscellaneous General Delays,0.013158,0,1
3,2014-01-02,11:30,Thursday,11,YU,YORK UNIVERSITY STATION,MUGD,Miscellaneous General Delays,0.013158,0,1
4,2014-01-02,11:30,Thursday,11,YU,FINCH WEST STATION,MUGD,Miscellaneous General Delays,0.013158,0,1


In [10]:
def get_n_dur(data):
    return pd.Series({'n_delays': data.indicator_ns.sum(), 'delay_duration': data.min_delay.sum()})
    
summ = df.groupby(['date', 'weekday', 'hour']).apply(get_n_dur).reset_index(drop = False)
summ.head()

,date,weekday,hour,n_delays,delay_duration
0,2014-01-01,Wednesday,0,1.0,55.0
1,2014-01-01,Wednesday,2,2.0,3.5
2,2014-01-01,Wednesday,3,3.0,8.5
3,2014-01-01,Wednesday,7,6.0,3.0
4,2014-01-01,Wednesday,8,7.0,34.0


# Any Weather Effects?

First, I wanted to investigate whether there are any weather effects

In [ ]:
weather = pd.read_csv('weather_data/daily_weather.csv', parse_dates = [4])

df_weather = (df.merge(weather[['Date/Time', 'Total Rain (mm)', 'Total Snow (cm)']], 
                       left_on = 'date', 
                       right_on = 'Date/Time', 
                       how = 'left')
                .rename(columns = {'Total Rain (mm)': 'rain_mm', 'Total Snow (cm)': 'snow_cm'})
                .drop(columns = 'Date/Time'))

In [ ]:
def get_summ_daily_weather(x):
    return pd.Series({'snow': x.snow_cm.mean(), 
                      'rain': x.rain_mm.mean(), 
                      'duration': x.min_delay.sum(), 
                      'n': x.indicator.sum()})

summ_weather = df_weather.groupby('date').apply(get_summ_daily_weather).sort_values(by = 'n', ascending = False)

In [ ]:
plt.scatter(summ_weather.n, summ_weather.snow)
# Days with a lot of snow or snow are not necessarily days with a lot of delays
# Can either adjust by removing some extreme weather days, or leave them in since there would always be days with bad weathers
# Adjusting for these would open up the questions of what other external/one-time causes to treat as outlier days

In [ ]:
# TO-DO: COMBINE THE PLOTS SIDE BY SIDE
plt.scatter(summ_weather.duration, summ_weather.snow)

In [ ]:
plt.scatter(summ_weather.n, summ_weather.rain)
# Snow/rain is not that correlated with amount of delay (perhaps because the subway is underground?)

In [ ]:
plt.scatter(summ_weather.duration, summ_weather.rain)

# Cause for Delays

In [ ]:
# Overall proportion of delay types each year, by frequency
delay_type_props = df.groupby('year').apply(lambda x: x.groupby('desc')['indicator'].sum() / x['indicator'].sum()).reset_index(drop = False)
delay_type_props.rename(columns = {'desc': 'delay_reason', 'indicator': 'prop'}, inplace = True)

In [ ]:
top_delays = delay_type_props.groupby('delay_reason')['prop'].sum().sort_values(ascending = False).index.values[:10]

plt.figure(figsize = (8, 5))
for delay in top_delays:
    plt.plot(delay_type_props[delay_type_props.delay_reason == delay].year, 
             delay_type_props[delay_type_props.delay_reason == delay].prop,
             label = delay)
plt.legend(bbox_to_anchor = (1, 0.8), loc="upper left")
# Still more ill passengers and disorderly patrons
# It seems the delays that occur frequent and delays that take a long time to resolve aren't the same
# Fires are resolved faster
# Disorderly patrons and PAAs seem to be taking longer

In [ ]:
# Overall proportion of delay types in each year, by duration
delay_type_props2 = df.groupby('year').apply(lambda x: x.groupby('desc')['min_delay'].sum() / x['min_delay'].sum()).reset_index(drop = False)
delay_type_props2.rename(columns = {'desc': 'delay_reason', 'min_delay': 'prop'}, inplace = True)

In [ ]:
top_delays2 = delay_type_props2.groupby('delay_reason')['prop'].sum().sort_values(ascending = False).index.values[:10]

plt.figure(figsize = (8, 5))
for delay in top_delays2:
    plt.plot(delay_type_props2[delay_type_props2.delay_reason == delay].year, 
             delay_type_props2[delay_type_props2.delay_reason == delay].prop,
             label = delay)
plt.legend(bbox_to_anchor = (1, 0.8), loc="upper left")
# Still more ill passengers and disorderly patrons
# It seems the delays that occur frequent and delays that take a long time to resolve aren't the same
# Fires are resolved faster
# Disorderly patrons and PAAs seem to be taking longer

In [ ]:
# Get count, mean and sd of delays by reason
def get_summ_n_mean_sd_sum(x):
    return pd.Series({'n': x.indicator.sum(),
                      'delay_mean': x.min_delay.mean(), 
                      'delay_sd': x.min_delay.std(), 
                      'delay_sum': x.min_delay.sum()})

summ_type = df.groupby('desc').apply(get_summ_n_mean_sd_sum).reset_index(drop = False)

In [ ]:
summ_type.sort_values(by = 'delay_sum', ascending = False)
# Some of the top factors leading to delays are outside TTC control
# except door problems seemed to be quite frequently occurring

In [ ]:
summ_type.sort_values(by = 'delay_mean', ascending = False)
# Delays that take a long time to resolve make sense - equipment problems, train hitting a person, fire
# Scheduled track maintenance and misc general delays - obvious data issues since they should not be 0min

In [ ]:
summ_type[summ_type.desc.isin(top_delays2)]
# Most time-consuming delays (by sum) are usually short delays but just very frequent
# The only exception is when train hits a person...

# By Day

In [ ]:
def get_summ_n_mean_sd_total(x):
    return pd.Series({'n': x.n.sum(), 
                      'delay_mean': x.duration.mean(), 
                      'delay_sd': x.duration.std(), 
                      'duration': x.duration.sum()})

summ_daily = summ.groupby('date').sum().groupby(pd.Grouper(freq = 'M')).apply(get_summ_n_mean_sd_total)

In [ ]:
plt.plot(summ_daily.index, summ_daily.delay_mean)
plt.fill_between(summ_daily.index, 
                 summ_daily.delay_mean - summ_daily.delay_sd, 
                 summ_daily.delay_mean + summ_daily.delay_sd, 
                 color = 'gray', alpha = 0.2)

# A bit of seasonality in the data (possibly correlated with ridership count)
# High at beginning of the year (Feb), spring (May, June), lowest in August (vacation so low ridership?), then back to previous level before leveling off
# Appears to have picked up since 2018
# which coincides with the opening of the Toronto-York subway extension that began service on Dec 17, 2017

## Year-over-Year

In [ ]:
daily_delay = summ.groupby('date').sum().drop(columns = ['hour']).reset_index(drop = False)
daily_delay.loc[:, 'year'] = list(map(lambda x: x.year, daily_delay.date))

In [ ]:
def subtract_years(dt, years = 1):
    try:
        dt = dt.replace(year = dt.year - years)
    except ValueError:
        dt = dt.replace(year = dt.year - years, day = dt.day - 1)
    return dt

# Look at year-over-year change
daily_delay.loc[:, 'date_py'] = list(map(subtract_years, daily_delay.date))

daily_delay = daily_delay[['date', 'date_py', 'n', 'duration']].merge(daily_delay[['date', 'n', 'duration']], 
                                                                      left_on = 'date_py', 
                                                                      right_on = 'date', 
                                                                      how = 'left', 
                                                                      suffixes = ['_cy', '_ly'])

In [ ]:
daily_delay.loc[:, 'yoy_chg_duration'] = daily_delay.duration_cy / daily_delay.duration_ly - 1
daily_delay.loc[:, 'yoy_chg_n'] = daily_delay.n_cy / daily_delay.n_ly - 1

In [ ]:
daily_delay.set_index('date_cy').groupby(pd.Grouper(freq = 'M'))['yoy_chg_duration'].mean().plot()
# Mostly yoy increase, although no linear trend observed

In [ ]:
daily_delay.set_index('date_cy').groupby(pd.Grouper(freq = 'M'))['yoy_chg_n'].mean().plot()
# The drastic spike in 2018 as new stations were added seems to have stabilized in 2019
# Large drop in 2019 could be because of high prior year base

# By Hour

In [ ]:
summ_hourly = summ[summ.date > '2017-12-31'].groupby('hour').apply(get_summ_n_mean_sd_total)

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('hour')
ax1.set_ylabel('# of delays')
ax1.plot(summ_hourly.index, summ_hourly.n, color = color)
ax1.tick_params(axis = 'y', labelcolor = color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('delay duration (minutes)')  # we already handled the x-label with ax1
ax2.plot(summ_hourly.index, summ_hourly.duration, color = color)
ax2.tick_params(axis = 'y', labelcolor = color)

fig.tight_layout()

# Similar pattern by count and total minutes of delay over course of a day
# Peaks when people are going to/from work, as expected
# Seems like delays in late afternoon/early evening take longer to resolve

In [ ]:
# Are there differences depending on which day of the week it is?
summ_hourly_weekly = (summ[summ.date > '2017-12-31'].groupby(['weekday', 'hour'])
                                                    .apply(get_summ_n_mean_sd_total)
                                                    .reset_index(drop = False))

summ_hourly_weekly.weekday = pd.Categorical(summ_hourly_weekly.weekday, 
                                            categories = ['Monday', 'Tuesday', 'Wednesday', 
                                                          'Thursday', 'Friday', 
                                                          'Saturday', 'Sunday'], 
                                            ordered = True) # Make sure facet grid is in the right chronological order below

In [ ]:
g = sns.FacetGrid(summ_hourly_weekly, col = 'weekday', col_wrap = 4)
g.map(plt.plot, "hour", "duration")
# Monday morning looks to be the worst of the whole week
# Friday afternoon looks to be the worst of the whole week
# The dip after midnight could be because the subway does not operate between 1:30am and 6am (currently) 

# By Day of Week

In [ ]:
def get_n_total(x):
    return pd.Series({'n': x.n.sum(), 'duration': x.duration.sum()})
    
summ_weekly = (summ[summ.date > '2017-12-31'].groupby(['weekday'])
                                             .apply(get_n_total))

In [ ]:
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

plt.bar(range(7), 
        summ_weekly.n.reindex(index = days_of_week),
        tick_label = days_of_week)
# As expected, there is a significant drop in delays on Saturday and Sundays
# The work week is fairly stable

In [ ]:
plt.bar(range(7), 
        summ_weekly.duration.reindex(index = days_of_week),
        tick_label = days_of_week)
# In terms of length of delays, Tuesday and Friday stick out more than the rest of the week
# Interesting that Wednesday is lowest...

# By Month

In [ ]:
# Which month is the worst for delays? 
# Maybe September (back from vacation) and Jan (back from vacation, bad weather)

summ_monthly = (summ[summ.date > '2017-12-31'].set_index('date')
                                              .groupby(pd.Grouper(freq = 'M'))
                                              .apply(get_n_total)
                                              .assign(month = lambda x: x.index.month)
                                              .groupby('month')
                                              .apply(get_n_total))

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('month')
ax1.set_ylabel('# of delays')
ax1.plot(summ_monthly.index, summ_monthly.n, color = color)
ax1.tick_params(axis = 'y', labelcolor = color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('delay duration (minutes)')  # we already handled the x-label with ax1
ax2.plot(summ_monthly.index, summ_monthly.duration, color = color)
ax2.tick_params(axis = 'y', labelcolor = color)

fig.tight_layout()

# More delays in Jan and Sept, but also interestingly in summer (albeit shorter ones on average)

# Stations

In [ ]:
# Which stations have the highest number of delays?
summ_station = (df[df.year > 2017].groupby(['station_fixed'])
                                  .apply(get_summ_n_mean_sd_sum)
                                  .reset_index(drop = False))

In [ ]:
summ_station.sort_values(by = 'n', ascending = False)[:10]

In [ ]:
summ_station.sort_values(by = 'delay_sum', ascending = False)[:10]
# Some stations are problematic by both measures

In [ ]:
# Try to visualize which stations are the hot spots for delays
# Disclaimer: map is not drawn to scale
station_coords = pd.read_csv('station_coords.csv')

station_coords = station_coords.merge(
    summ_station, 
    left_on = 'station', 
    right_on = 'station_fixed', 
    how = 'left'
)

station_coords.loc[:, 'text'] = list(map(lambda x, y, z, k: f"{x}<br># delays: {round(k)}<br>Mean delay: {round(y, 2)}<br>Stdev delay: {round(z, 2)}",
                                         station_coords.station, 
                                         station_coords.delay_mean, 
                                         station_coords.delay_sd,
                                         station_coords.n))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

cmap = {'YU': 'yellow', 'BD': 'green', 'SRT': 'blue', 'SHP': 'magenta'}

fig = go.Figure()

fig.add_trace(go.Scatter(
    x = station_coords[station_coords.line.isin(['BD', 'SRT'])]['xcoord'],
    y = station_coords[station_coords.line.isin(['BD', 'SRT'])]['ycoord'],
    mode = 'lines',
    line = dict(color = 'lightslategrey'),
    showlegend = False
))

fig.add_trace(go.Scatter(
    x = station_coords[station_coords.line.isin(['YU'])]['xcoord'],
    y = station_coords[station_coords.line.isin(['YU'])]['ycoord'],
    mode = 'lines',
    line = dict(color = 'lightslategrey'),
    showlegend = False
))

fig.add_trace(go.Scatter(
    x = station_coords[(station_coords.line.isin(['SHP'])) | (station_coords.station_fixed == 'SHEPPARD STATION')]['xcoord'],
    y = station_coords[(station_coords.line.isin(['SHP'])) | (station_coords.station_fixed == 'SHEPPARD STATION')]['ycoord'],
    mode = 'lines',
    line = dict(color = 'lightslategrey'),
    showlegend = False
))

for name, group in station_coords.groupby('line'):
    fig.add_trace(go.Scatter(
        x = group['xcoord'],
        y = group['ycoord'],
        mode = 'markers',
        marker = dict(
            size = group['delay_sum'] / 100, 
            color = cmap[name],
            line = dict(width = 1, color = 'darkslategrey')
        ),
        text = station_coords['text'],
        hoverinfo = 'text',
        name = name
    ))
    
fig.update_layout(
    legend = go.layout.Legend(
        itemsizing = 'constant'
    ),
    plot_bgcolor = 'white'
)

fig.update_xaxes(visible = False)
fig.update_yaxes(visible = False)

fig.show()

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

station_coords = pd.read_csv('station_coords.csv')

@interact
def plot_subway_delays(
    Month = widgets.IntSlider(min = 1, max = 12, step = 1, value = 11),
    Weekday = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'],
    Hour = widgets.IntSlider(min = 0, max = 23, step = 1, value = 7)):
    
    data_sub = df[(df.year > 2017) & (df.hour == Hour) & (df.month == Month) & (df.weekday == Weekday)]
    summ_station = (data_sub.groupby(['station_fixed'])
                            .apply(get_summ_n_mean_sd_sum)
                            .reset_index(drop = False))
    
    data_plot = station_coords.merge(
        summ_station, 
        left_on = 'station', 
        right_on = 'station_fixed', 
        how = 'left'
    ).fillna(0)
    
    data_plot.loc[:, 'text'] = list(map(lambda x, y, z, k: f"{x}<br># delays: {round(k)}<br>Mean delay: {round(y, 2)}<br>Stdev delay: {round(z, 2)}",
                                        data_plot.station, 
                                        data_plot.delay_mean, 
                                        data_plot.delay_sd,
                                        data_plot.n))
    
    cmap = {'YU': 'yellow', 'BD': 'green', 'SRT': 'blue', 'SHP': 'magenta'}

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x = data_plot[data_plot.line.isin(['BD', 'SRT'])]['xcoord'],
        y = data_plot[data_plot.line.isin(['BD', 'SRT'])]['ycoord'],
        mode = 'lines',
        line = dict(color = 'lightslategrey'),
        showlegend = False
    ))

    fig.add_trace(go.Scatter(
        x = data_plot[data_plot.line.isin(['YU'])]['xcoord'],
        y = data_plot[data_plot.line.isin(['YU'])]['ycoord'],
        mode = 'lines',
        line = dict(color = 'lightslategrey'),
        showlegend = False
    ))

    fig.add_trace(go.Scatter(
        x = data_plot[(data_plot.line.isin(['SHP'])) | (data_plot.station == 'SHEPPARD STATION')]['xcoord'],
        y = data_plot[(data_plot.line.isin(['SHP'])) | (data_plot.station == 'SHEPPARD STATION')]['ycoord'],
        mode = 'lines',
        line = dict(color = 'lightslategrey'),
        showlegend = False
    ))

    for name, group in data_plot.groupby('line'):
        fig.add_trace(go.Scatter(
            x = group['xcoord'],
            y = group['ycoord'],
            mode = 'markers',
            marker = dict(
                size = 2*group['delay_sum'] + 5, 
                color = cmap[name],
                line = dict(width = 1, color = 'darkslategrey')
            ),
            text = group['text'],
            hoverinfo = 'text',
            name = name
        ))

    fig.update_layout(
        legend = go.layout.Legend(
            itemsizing = 'constant'
        ),
        plot_bgcolor = 'white'
    )

    fig.update_xaxes(visible = False)
    fig.update_yaxes(visible = False)

    fig.show()

In [ ]:
c_map = {'YU': 'yellow', 'BD': 'green', 'SRT': 'blue', 'SHP': 'magenta'}

plt.figure(figsize = (13, 7))
for name, group in station_coords.groupby('line'):
    plt.scatter(group.xcoord, group.ycoord, c = c_map[name], s = group.delay_sum / 10)

yu_legend = mlines.Line2D([], [], color = 'yellow', marker = 'o', linestyle = 'None',
                          markersize = 10, label = 'Yonge-University')
bd_legend = mlines.Line2D([], [], color = 'green', marker = 'o', linestyle = 'None',
                          markersize = 10, label = 'Bloor-Danforth')
srt_legend = mlines.Line2D([], [], color = 'blue', marker = 'o', linestyle = 'None',
                           markersize = 10, label = 'Scarborough RT')
shp_legend = mlines.Line2D([], [], color = 'magenta', marker = 'o', linestyle = 'None',
                           markersize = 10, label = 'Sheppard')
plt.legend(handles = [yu_legend, bd_legend, srt_legend, shp_legend], 
           bbox_to_anchor = (1, 0.8), 
           loc = "upper left")

for _, row in station_coords.sort_values(by = 'delay_sum', ascending = False)[:10].iterrows():
    plt.annotate(re.sub(' STATION', '', row['station_fixed']), 
                 xy = (row['xcoord'], row['ycoord']), 
                 xytext = (row['xcoord'] + 1, row['ycoord'] - 1), 
                 arrowprops = dict(arrowstyle = "->", connectionstyle = "arc3"))

plt.title('Total Delays by Station')
# All final stops have long delays, perhaps because there is more rider traffic at those stops

In [ ]:
c_map = {'YU': 'yellow', 'BD': 'green', 'SRT': 'blue', 'SHP': 'magenta'}

plt.figure(figsize = (13, 7))
for name, group in station_coords.groupby('line'):
    plt.scatter(group.xcoord, group.ycoord, c = c_map[name], s = group.delay_mean * 30)
    
plt.legend(handles = [yu_legend, bd_legend, srt_legend, shp_legend], 
           bbox_to_anchor = (1, 0.8), 
           loc = "upper left")

for _, row in station_coords.sort_values(by = 'delay_mean', ascending = False)[:10].iterrows():
    plt.annotate(re.sub(' STATION', '', row['station_fixed']), 
                 xy = (row['xcoord'], row['ycoord']), 
                 xytext = (row['xcoord'] + 1, row['ycoord'] - 1), 
                 arrowprops = dict(arrowstyle = "->", connectionstyle = "arc3"))

plt.title('Average Delays by Station')
# The average duration of delay events is highest on the Scarborough RT line, which is coincidentally above ground
# Maybe it is harder to maintain the tracks? Or more weather-related damage?

## Delay Type by Line

In [ ]:
summ_line_type = df[(df.year > 2017) & 
                    (df.line_cleaned.isin(['YU', 'BD', 'SRT', 'SHP']))].groupby(['line_cleaned', 'desc']).apply(get_summ_n_mean_sd_sum)

In [ ]:
summ_line_type.groupby('line_cleaned').apply(lambda x: x.sort_values(by = 'delay_sum', ascending = False)[:10])
# Top reasons for delay by line
# Showing weather related delays play a large factor for SRT but not for the other lines that are predominantly underground
# A lot of passenger-related problems on YU and BD lines